# Subsidence Risk Analysis for Bristol

This notebook performs geospatial analysis to assess **subsidence risk for individual buildings** based on:

- **🌳 Tree Data** - From Bristol City Council API (crown width, species, location)
- **🏠 Building Data** - From OpenStreetMap via Overpass API  
- **🌍 Soil Data** - From British Geological Survey (BGS) WMS service

## How It Works

The analysis calculates a **risk score (0-10) for each building** by combining:

1. **Soil Risk (40%)** - Samples BGS soil texture raster at building location
   - Heavy clay soils = highest risk (shrink-swell potential)
   - Sandy soils = lowest risk (stable)
   - Unknown/no-data areas = score of 0 (excluded from risk)

2. **Tree Proximity Risk (60%)** - Weighted distance calculation
   - Trees within root spread zone = high influence  
   - Root spread estimated as 1.5× crown width
   - Species-specific risk factors (willow > oak > birch)
   - Very close trees (< 5m) get bonus scoring

## Outputs

- `bristol_buildings_scored.geojson` - All buildings with risk scores and attributes
- `subsidence_risk_buildings.html` - Interactive map with toggleable layers

## 1. Setup and Imports

Import required libraries for geospatial analysis, visualization, and API calls.

In [77]:
import geopandas as gpd
import pandas as pd
import numpy as np
import folium
from folium import plugins
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import osmnx as ox
from shapely.geometry import Point, Polygon, box
from IPython.display import display, IFrame
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)

# Set plotting style with fallback
try:
    plt.style.use('seaborn-v0_8-darkgrid')
except:
    try:
        plt.style.use('seaborn-darkgrid')
    except:
        plt.style.use('default')

print("Libraries imported successfully!")

Libraries imported successfully!


## 2. Study Area Configuration

Define the Bristol study area bounds. Three preset areas are available:
- **TINY** (~0.25 km²) - Quick testing
- **TEST** (~4 km²) - Development  
- **FULL** (~180 km²) - Full Bristol analysis

In [78]:
# Bristol city center coordinates
BRISTOL_CENTER = (51.4545, -2.5879)

# FULL STUDY AREA (use this for final analysis)
BRISTOL_BOUNDS_FULL = {
    'north': 51.5200,
    'south': 51.4000,
    'east': -2.5000,
    'west': -2.7000
}

# SMALL TEST AREA (use this for testing - ~2km x 2km around city center)
BRISTOL_BOUNDS_TEST = {
    'north': 51.4645,  # ~1km north of center
    'south': 51.4445,  # ~1km south of center
    'east': -2.5779,   # ~1km east of center
    'west': -2.5979    # ~1km west of center
}

# TINY TEST AREA (for quick testing - ~500m x 500m) - RECOMMENDED FOR FIRST RUN
BRISTOL_BOUNDS_TINY = {
    'north': 51.4595,  # ~500m north of center
    'south': 51.4495,  # ~500m south of center
    'east': -2.5829,   # ~500m east of center
    'west': -2.5929    # ~500m west of center
}

# Switch between test areas (BRISTOL_BOUNDS_TINY is fastest for testing)
BRISTOL_BOUNDS = BRISTOL_BOUNDS_FULL

# Create bounding box geometry
bbox_geometry = box(
    BRISTOL_BOUNDS['west'],
    BRISTOL_BOUNDS['south'],
    BRISTOL_BOUNDS['east'],
    BRISTOL_BOUNDS['north']
)

## 3. Risk Scoring Configuration

All scoring parameters are defined in a single location for easy adjustment.

### How Risk is Calculated

Each building receives a **combined risk score (0-10)** calculated as:

```
Combined Score = (Soil Score × 0.4) + (Tree Score × 0.6)
```

#### Soil Score (40% weight)
Based on soil type at building location. Clay soils shrink when dry and swell when wet, causing ground movement.

| Soil Type | Score | Why |
|-----------|-------|-----|
| Heavy Clay | 10 | Extreme shrink-swell |
| Clay | 8 | High shrink-swell |
| Clay Loam | 6 | Moderate shrink-swell |
| Loam | 4 | Some movement |
| Sandy Loam | 2 | Minimal movement |
| Sand | 1 | Stable |
| Unknown | 0 | No data - excluded |

#### Tree Score (60% weight)
Based on proximity to trees. Tree roots extract water from soil, causing clay shrinkage.

**Scoring approach:**
- Trees within root zone (1.5× crown width): **3-5 points** per tree
- Very close trees (< 5m): **+2 point bonus**
- Trees outside root zone: Linear decay to 0 at 30m

**Species Risk Factors:**
- 🌳 Willow: 1.8× (very high water uptake)
- 🌳 Poplar: 1.7×
- 🌳 Oak: 1.5×
- 🌳 Ash: 1.3×
- 🌳 Default: 1.0×

### Worked Examples

**Example 1: High Risk Building**
- Soil: Heavy Clay (score = 10)
- Trees: 2 willows within 5m (tree score = 9.0)
- **Combined: (10 × 0.4) + (9.0 × 0.6) = 4.0 + 5.4 = 9.4** ⚠️ HIGH

**Example 2: Low Risk Building**
- Soil: Sandy Loam (score = 2)
- Trees: 1 birch at 20m (tree score = 0.5)
- **Combined: (2 × 0.4) + (0.5 × 0.6) = 0.8 + 0.3 = 1.1** ✅ LOW

**Example 3: Tree-Dominated Risk**
- Soil: Loam (score = 4)  
- Trees: 3 oaks within root zone (tree score = 8.0)
- **Combined: (4 × 0.4) + (8.0 × 0.6) = 1.6 + 4.8 = 6.4** 🟡 MEDIUM

In [99]:
# ============================================
# RISK SCORING CONFIGURATION
# ============================================
# All scoring parameters in one place for easy adjustment

# ----- RISK WEIGHTING -----
# How much each factor contributes to final score
RISK_WEIGHTS = {
    'soil': 0.4,    # 40% - Soil type is primary driver
    'tree': 0.6     # 60% - Tree proximity is secondary
}

# ----- SOIL RISK SCORES -----
# Score based on shrink-swell potential (0-10 scale)
SOIL_RISK_SCORES = {
    'Heavy Clay': 10.0,   # Highest risk - extreme shrink-swell
    'Clay': 8.0,          # High risk
    'Clay Loam': 6.0,     # Moderate-high risk
    'Loam': 4.0,          # Moderate risk
    'Sandy Loam': 2.0,    # Low risk
    'Sand': 1.0,          # Minimal risk - stable
    'Unknown': 0.0        # No BGS data - doesn't contribute to score
}

# Soil raster code to score mapping (from SoilRasterHandler classification)
# Unknown (0) gets score of 0 so buildings without BGS data don't get inflated scores
SOIL_CODE_SCORES = {
    6: 10.0,  # Heavy Clay
    5: 8.0,   # Clay
    4: 6.0,   # Clay Loam
    3: 4.0,   # Loam
    2: 2.0,   # Sandy Loam
    1: 1.0,   # Sand
    0: 0.0    # Unknown - no soil data, no contribution to score
}

# ----- TREE RISK CONFIGURATION -----
TREE_CONFIG = {
    'max_influence_distance_m': 30.0,    # Trees beyond this don't contribute
    'root_spread_multiplier': 1.5,       # Root spread = crown_width × this (typical rule of thumb)
    'default_crown_width_m': 8.0,        # Default if data missing
    'distance_decay_power': 1.0,         # Linear decay (was 2.0 - too aggressive)
    'max_trees_to_consider': 50,         # Performance limit
    'base_tree_score': 3.0,              # Score for single tree within root zone
    'close_tree_bonus': 2.0,             # Extra score for trees < 5m away
    'score_cap': 10.0,                   # Maximum tree score
}

# Species risk factors - some trees extract more water
# Values > 1.0 increase risk, < 1.0 decrease risk
SPECIES_RISK_FACTORS = {
    'willow': 1.8,       # Very high water uptake
    'poplar': 1.7,       # High water uptake
    'oak': 1.5,          # Moderate-high
    'ash': 1.3,          # Moderate
    'sycamore': 1.2,
    'lime': 1.1,
    'cherry': 1.0,
    'birch': 0.9,        # Lower water uptake
    'pine': 0.7,         # Evergreen, lower risk
    'default': 1.0       # Unknown species
}

# ----- RISK CATEGORIES -----
# Score thresholds for categorization
RISK_THRESHOLDS = {
    'high': 7.0,      # Score >= 7 is HIGH risk
    'medium': 4.0,    # Score >= 4 is MEDIUM risk
    # Below 4 is LOW risk
}

# Print configuration summary
print("=" * 60)
print("RISK SCORING CONFIGURATION")
print("=" * 60)
print(f"\n📊 Risk Weights:")
print(f"   Soil:  {RISK_WEIGHTS['soil']*100:.0f}%")
print(f"   Trees: {RISK_WEIGHTS['tree']*100:.0f}%")

print(f"\n🌍 Soil Risk Scores:")
for soil, score in SOIL_RISK_SCORES.items():
    if soil != 'Unknown':
        bar = '█' * int(score)
        print(f"   {soil:12s}: {score:4.1f} {bar}")

print(f"\n🌳 Tree Configuration:")
print(f"   Max influence distance: {TREE_CONFIG['max_influence_distance_m']}m")
print(f"   Root spread: {TREE_CONFIG['root_spread_multiplier']}× crown width")

print(f"\n🌳 Species Risk Factors (top 5):")
sorted_species = sorted(SPECIES_RISK_FACTORS.items(), key=lambda x: x[1], reverse=True)
for species, factor in sorted_species[:5]:
    print(f"   {species:10s}: {factor:.1f}×")

print(f"\n⚠️ Risk Categories:")
print(f"   HIGH:   score ≥ {RISK_THRESHOLDS['high']}")
print(f"   MEDIUM: score ≥ {RISK_THRESHOLDS['medium']}")
print(f"   LOW:    score < {RISK_THRESHOLDS['medium']}")

RISK SCORING CONFIGURATION

📊 Risk Weights:
   Soil:  40%
   Trees: 60%

🌍 Soil Risk Scores:
   Heavy Clay  : 10.0 ██████████
   Clay        :  8.0 ████████
   Clay Loam   :  6.0 ██████
   Loam        :  4.0 ████
   Sandy Loam  :  2.0 ██
   Sand        :  1.0 █

🌳 Tree Configuration:
   Max influence distance: 30.0m
   Root spread: 1.5× crown width

🌳 Species Risk Factors (top 5):
   willow    : 1.8×
   poplar    : 1.7×
   oak       : 1.5×
   ash       : 1.3×
   sycamore  : 1.2×

⚠️ Risk Categories:
   HIGH:   score ≥ 7.0
   MEDIUM: score ≥ 4.0
   LOW:    score < 4.0


## 4. Data Collection

### 4.1 Tree Data (Bristol City Council API)

Fetches the complete tree inventory from Bristol City Council's open data API:
- Location (point geometry in WGS84)
- Crown width, height, and area measurements
- Species information (common and Latin names)
- Diameter at breast height (DBH)

**Source:** Bristol City Council GIS MapServer

In [72]:
def collect_tree_data(north, south, east, west):
    """
    Collect tree data from Bristol City Council API.
    
    Parameters:
    -----------
    north, south, east, west : float
        Bounding box coordinates (used for filtering if needed)
    
    Returns:
    --------
    geopandas.GeoDataFrame
        Tree features from Bristol API
    """
    try:
        # Bristol tree data API endpoint (using GeoJSON format)
        api_url = "https://maps2.bristol.gov.uk/server2/rest/services/ext/ll_environment_and_planning/MapServer/32/query"
        
        print("Fetching tree data from Bristol City Council API...")
        
        # First, get the total count
        count_params = {
            'where': '1=1',
            'returnCountOnly': 'true',
            'f': 'json'
        }
        
        count_response = requests.get(api_url, params=count_params, timeout=30)
        count_response.raise_for_status()
        total_count = count_response.json().get('count', 0)
        print(f"Total records available: {total_count}")
        
        # Fetch all records using pagination
        all_features = []
        batch_size = 1000  # API limit per request
        offset = 0
        
        while offset < total_count:
            params = {
                'where': '1=1',
                'outFields': '*',
                'f': 'geojson',
                'resultOffset': offset,
                'resultRecordCount': batch_size
            }
            
            print(f"Fetching records {offset} to {min(offset + batch_size, total_count)}...")
            response = requests.get(api_url, params=params, timeout=60)
            response.raise_for_status()
            
            data = response.json()
            
            if 'features' in data and len(data['features']) > 0:
                all_features.extend(data['features'])
                offset += len(data['features'])
            else:
                break
        
        print(f"Successfully fetched {len(all_features)} total features")
        
        # Convert all features to GeoDataFrame
        if len(all_features) > 0:
            # Create a GeoJSON structure
            geojson_data = {
                'type': 'FeatureCollection',
                'features': all_features
            }
            
            # Create GeoDataFrame directly from GeoJSON
            trees_gdf = gpd.GeoDataFrame.from_features(geojson_data, crs='EPSG:4326')
            
            # Filter to bounding box
            trees_gdf = trees_gdf.cx[west:east, south:north]
            
            print(f"Collected {len(trees_gdf)} tree features within study area")
            print(f"Available columns: {trees_gdf.columns.tolist()}")
            return trees_gdf
        else:
            print("No tree features found in API response")
            return gpd.GeoDataFrame()
    
    except Exception as e:
        print(f"Error collecting tree data: {e}")
        import traceback
        traceback.print_exc()
        # Return empty GeoDataFrame if collection fails
        return gpd.GeoDataFrame()

# Collect tree data
tree_data = collect_tree_data(
    BRISTOL_BOUNDS['north'],
    BRISTOL_BOUNDS['south'],
    BRISTOL_BOUNDS['east'],
    BRISTOL_BOUNDS['west']
)

if not tree_data.empty:
    print(f"\nTree data shape: {tree_data.shape}")
    print(f"Tree data columns: {tree_data.columns.tolist()}")
    print(f"\nSample tree data:")
    print(tree_data.head())
    
    # Check geometry type
    print(f"\nGeometry types: {tree_data.geometry.type.unique()}")
else:
    print("\nNo tree data available")

Fetching tree data from Bristol City Council API...
Total records available: 55359
Fetching records 0 to 1000...
Total records available: 55359
Fetching records 0 to 1000...
Fetching records 1000 to 2000...
Fetching records 1000 to 2000...
Fetching records 2000 to 3000...
Fetching records 2000 to 3000...
Fetching records 3000 to 4000...
Fetching records 3000 to 4000...
Fetching records 4000 to 5000...
Fetching records 4000 to 5000...
Fetching records 5000 to 6000...
Fetching records 5000 to 6000...
Fetching records 6000 to 7000...
Fetching records 6000 to 7000...
Fetching records 7000 to 8000...
Fetching records 7000 to 8000...
Fetching records 8000 to 9000...
Fetching records 8000 to 9000...
Fetching records 9000 to 10000...
Fetching records 9000 to 10000...
Fetching records 10000 to 11000...
Fetching records 10000 to 11000...
Fetching records 11000 to 12000...
Fetching records 11000 to 12000...
Fetching records 12000 to 13000...
Fetching records 12000 to 13000...
Fetching records 130

### 4.2 Building Data (OpenStreetMap)

Fetches building footprints from OpenStreetMap via Overpass API.

**Performance by study area size:**
- TINY (~0.25 km²): ~10-30 seconds
- TEST (~4 km²): ~1-3 minutes  
- FULL (~180 km²): ~5-15 minutes

**Note:** For large areas, data is fetched directly from Overpass API mirrors with automatic failover.

In [59]:
def collect_building_data(north, south, east, west, skip=False, method='overpass_direct'):
    """
    Collect building data from OpenStreetMap.
    
    Parameters:
    -----------
    north, south, east, west : float
        Bounding box coordinates
    skip : bool
        If True, skip building data collection and return empty GeoDataFrame
    method : str
        'overpass_direct' - Direct Overpass API calls (fastest, most reliable)
        'osmnx' - Use OSMnx library (slower fallback)
    
    Returns:
    --------
    geopandas.GeoDataFrame
        Building features
    """
    if skip:
        print("Skipping building data collection (skip=True)")
        print("Analysis will continue without building density factor")
        return gpd.GeoDataFrame()
    
    # Calculate approximate area
    area_deg = abs(north - south) * abs(east - west)
    area_km = area_deg * 111 * 111
    print(f"Fetching buildings for area: ~{area_km:.2f} km²")
    print(f"Bounding box: N={north:.4f}, S={south:.4f}, E={east:.4f}, W={west:.4f}")
    
    # Method 1: Direct Overpass API (RECOMMENDED - faster and more reliable)
    if method == 'overpass_direct':
        try:
            print("\n[Method 1] Using Overpass API directly...")
            
            # List of Overpass API mirrors for redundancy
            overpass_urls = [
                "https://overpass-api.de/api/interpreter",
                "https://overpass.kumi.systems/api/interpreter",
                "https://overpass.openstreetmap.ru/api/interpreter"
            ]
            
            # Optimized Overpass QL query - gets only essential data
            query = f"""
            [out:json][timeout:60];
            (
              way["building"]({south},{west},{north},{east});
              relation["building"]({south},{west},{north},{east});
            );
            out geom;
            """
            
            buildings_data = None
            
            for api_url in overpass_urls:
                try:
                    print(f"  Trying {api_url.split('/')[2]}...")
                    response = requests.post(
                        api_url, 
                        data=query, 
                        timeout=90,
                        headers={'User-Agent': 'SubsidenceRiskAnalysis/1.0'}
                    )
                    
                    if response.status_code == 200:
                        buildings_data = response.json()
                        print(f"  ✓ Success!")
                        break
                    else:
                        print(f"  ✗ HTTP {response.status_code}")
                        continue
                        
                except requests.exceptions.Timeout:
                    print(f"  ✗ Timeout")
                    continue
                except Exception as e:
                    print(f"  ✗ Error: {str(e)[:50]}")
                    continue
            
            if buildings_data and 'elements' in buildings_data:
                elements = buildings_data['elements']
                print(f"\n✓ Retrieved {len(elements)} building elements from Overpass API")
                
                # Convert Overpass JSON to GeoDataFrame
                features = []
                for element in elements:
                    if 'geometry' in element:
                        # Extract coordinates
                        coords = [(node['lon'], node['lat']) for node in element['geometry']]
                        if len(coords) >= 3:  # Valid polygon needs at least 3 points
                            from shapely.geometry import Polygon
                            poly = Polygon(coords)
                            features.append({
                                'geometry': poly,
                                'building': element.get('tags', {}).get('building', 'yes'),
                                'osm_id': element.get('id'),
                                'building_levels': element.get('tags', {}).get('building:levels'),
                                'name': element.get('tags', {}).get('name')
                            })
                
                if features:
                    buildings_gdf = gpd.GeoDataFrame(features, crs='EPSG:4326')
                    print(f"✓ Created GeoDataFrame with {len(buildings_gdf)} buildings")
                    return buildings_gdf
                else:
                    print("✗ No valid building polygons found")
                    return gpd.GeoDataFrame()
            else:
                print("✗ All Overpass API mirrors failed or returned no data")
                print("  Falling back to OSMnx method...")
                method = 'osmnx'
                
        except Exception as e:
            print(f"✗ Overpass direct method failed: {e}")
            print("  Falling back to OSMnx method...")
            method = 'osmnx'
    
    # Method 2: OSMnx (fallback, slower but sometimes works)
    if method == 'osmnx':
        try:
            print("\n[Method 2] Using OSMnx library...")
            print("  Note: This can take 5-20+ minutes for dense urban areas")
            
            buildings_gdf = ox.features_from_bbox(
                bbox=(north, south, east, west),
                tags={'building': True}
            )
            
            print(f"✓ Collected {len(buildings_gdf)} building features")
            return buildings_gdf
            
        except Exception as e:
            print(f"✗ OSMnx method failed: {e}")
    
    print("\n✗ All methods failed to retrieve building data")
    print("\nRECOMMENDATIONS:")
    print("1. Set skip=True to run analysis without buildings")
    print("2. Load pre-collected data: gpd.read_file('bristol_buildings_tiny.geojson')")
    print("3. Try again later when Overpass API is less busy")
    
    return gpd.GeoDataFrame()

# ============================================
# OPTION 1: Load pre-collected data (RECOMMENDED - instant, proven to work)
# ============================================
# Uncomment to use pre-collected building data:
# building_data = gpd.read_file('bristol_buildings_tiny.geojson')
# print(f"✓ Loaded {len(building_data)} buildings from file")

# ============================================
# OPTION 2: Collect from API in notebook (may timeout in notebook environment)
# ============================================
# Comment out if using Option 1 above
SKIP_BUILDINGS = False  # Set True to skip building collection entirely

building_data = collect_building_data(
    BRISTOL_BOUNDS['north'],
    BRISTOL_BOUNDS['south'],
    BRISTOL_BOUNDS['east'],
    BRISTOL_BOUNDS['west'],
    skip=SKIP_BUILDINGS,
    method='overpass_direct'  # Options: 'overpass_direct', 'osmnx'
)

if not building_data.empty:
    print(f"\nBuilding data shape: {building_data.shape}")
    print(f"Building data columns: {building_data.columns.tolist()[:10]}...")

Fetching buildings for area: ~295.70 km²
Bounding box: N=51.5200, S=51.4000, E=-2.5000, W=-2.7000

[Method 1] Using Overpass API directly...
  Trying overpass-api.de...
  ✓ Success!

✓ Retrieved 227769 building elements from Overpass API
  ✓ Success!

✓ Retrieved 227769 building elements from Overpass API
✓ Created GeoDataFrame with 227578 buildings

Building data shape: (227578, 5)
Building data columns: ['geometry', 'building', 'osm_id', 'building_levels', 'name']...
✓ Created GeoDataFrame with 227578 buildings

Building data shape: (227578, 5)
Building data columns: ['geometry', 'building', 'osm_id', 'building_levels', 'name']...


### 4.3 Soil Data (British Geological Survey WMS)

Fetches soil texture data from the BGS UK Soil Observatory WMS service.

**Data source:** `Parent.Material.Soil.texture` layer from BGS

**How it works:**
1. Area is split into a grid of tiles (~1.6km × 1.5km each)
2. Each tile is fetched separately to handle large areas
3. Tiles are stitched into a single raster
4. Soil types are classified from raster colors:
   - Light colors → Sandy soils (low risk)
   - Dark/brown colors → Clay soils (high risk)

**Note:** Areas where BGS returns no data are marked as "Unknown" and receive a soil score of 0.

In [79]:
# ============================================
# SOIL DATA HANDLER (with tiled fetching)
# ============================================

class SoilRasterHandler:
    """Handles BGS WMS soil data as a raster layer with tiled fetching for large areas."""
    
    def __init__(self, bounds, resolution=512, tile_size=128):
        """
        Initialize the soil raster handler.
        
        Parameters:
        -----------
        bounds : dict
            Dictionary with 'north', 'south', 'east', 'west' keys
        resolution : int
            Total pixel resolution for the final raster image
        tile_size : int
            Size of each tile when fetching (smaller = more requests but better coverage)
        """
        self.bounds = bounds
        self.resolution = resolution
        self.tile_size = tile_size
        self.raster_array = None
        self.transform = None
        self.soil_type_map = None
        
    def fetch_wms_raster(self, use_real_data=False):
        """
        Fetch soil data from BGS WMS and store as raster.
        Uses tiled fetching to handle large areas that exceed WMS limits.
        
        Returns:
        --------
        numpy.ndarray
            Soil raster array
        """
        if not use_real_data:
            print("Generating synthetic soil raster (set use_real_data=True for BGS data)")
            return self._generate_synthetic_raster()
        
        try:
            from owslib.wms import WebMapService
            from PIL import Image
            from io import BytesIO
            
            print("Fetching soil raster from BGS WMS (tiled mode)...")
            
            wms_url = 'https://map.bgs.ac.uk/arcgis/services/UKSO/UKSO_BGS/MapServer/WMSServer'
            wms = WebMapService(wms_url, version='1.3.0')
            
            # Find a queryable layer
            preferred_layers = [
                'Parent.Material.Soil.texture',
                'Parent.Material.Soil.Wash',
                'Peat.Coverage',
                'Parent.Material.Grain.size',
            ]
            
            layer_name = None
            for candidate in preferred_layers:
                if candidate in wms.contents:
                    layer_obj = wms.contents[candidate]
                    if getattr(layer_obj, 'queryable', 0) == 1:
                        layer_name = candidate
                        break
            
            if not layer_name:
                raise Exception("No queryable layers found in WMS service")
            
            print(f"  Layer: {layer_name}")
            
            # Calculate tile grid
            width_deg = self.bounds['east'] - self.bounds['west']
            height_deg = self.bounds['north'] - self.bounds['south']
            
            # Determine optimal tile count based on area size
            # BGS WMS seems to have issues with requests covering > ~0.02 degrees
            max_tile_deg = 0.015  # Max degrees per tile dimension
            
            n_tiles_x = max(1, int(np.ceil(width_deg / max_tile_deg)))
            n_tiles_y = max(1, int(np.ceil(height_deg / max_tile_deg)))
            
            # Calculate tile dimensions in degrees
            tile_width_deg = width_deg / n_tiles_x
            tile_height_deg = height_deg / n_tiles_y
            
            # Calculate pixels per tile
            pixels_per_tile_x = self.resolution // n_tiles_x
            pixels_per_tile_y = self.resolution // n_tiles_y
            
            print(f"  Grid: {n_tiles_x}×{n_tiles_y} tiles ({n_tiles_x * n_tiles_y} total requests)")
            print(f"  Tile size: ~{tile_width_deg*111:.1f}km × {tile_height_deg*111:.1f}km")
            
            # Initialize combined raster
            combined_raster = np.zeros((self.resolution, self.resolution, 4), dtype=np.uint8)
            combined_raster[:, :, 3] = 0  # Start fully transparent
            
            tiles_with_data = 0
            tiles_empty = 0
            
            # Fetch each tile
            for ty in range(n_tiles_y):
                for tx in range(n_tiles_x):
                    # Calculate tile bounds
                    tile_west = self.bounds['west'] + tx * tile_width_deg
                    tile_east = tile_west + tile_width_deg
                    tile_north = self.bounds['north'] - ty * tile_height_deg
                    tile_south = tile_north - tile_height_deg
                    
                    tile_bbox = (tile_west, tile_south, tile_east, tile_north)
                    
                    try:
                        img = wms.getmap(
                            layers=[layer_name],
                            srs='EPSG:4326',
                            bbox=tile_bbox,
                            size=(pixels_per_tile_x, pixels_per_tile_y),
                            format='image/png',
                            transparent=True
                        )
                        
                        img_data = Image.open(BytesIO(img.read()))
                        tile_array = np.array(img_data)
                        
                        # Ensure RGBA format
                        if tile_array.shape[-1] == 3:
                            alpha = np.full(tile_array.shape[:2] + (1,), 255, dtype=np.uint8)
                            tile_array = np.concatenate([tile_array, alpha], axis=-1)
                        
                        # Check if tile has real data (non-transparent)
                        has_data = (tile_array[:, :, 3] > 0).any()
                        
                        if has_data:
                            tiles_with_data += 1
                        else:
                            tiles_empty += 1
                        
                        # Calculate position in combined raster
                        y_start = ty * pixels_per_tile_y
                        y_end = y_start + pixels_per_tile_y
                        x_start = tx * pixels_per_tile_x
                        x_end = x_start + pixels_per_tile_x
                        
                        # Handle edge tiles that might be slightly different size
                        actual_h, actual_w = tile_array.shape[:2]
                        y_end = min(y_end, y_start + actual_h)
                        x_end = min(x_end, x_start + actual_w)
                        
                        combined_raster[y_start:y_end, x_start:x_end] = tile_array[:y_end-y_start, :x_end-x_start]
                        
                    except Exception as e:
                        tiles_empty += 1
                        # Leave tile as transparent
                
                # Progress indicator
                progress = ((ty * n_tiles_x + tx + 1) / (n_tiles_x * n_tiles_y)) * 100
                print(f"\r  Progress: {progress:.0f}% ({tiles_with_data} tiles with data)", end="")
            
            print()  # New line after progress
            
            self.raster_array = combined_raster
            
            # Create geotransform for coordinate lookup
            self.transform = {
                'west': self.bounds['west'],
                'north': self.bounds['north'],
                'pixel_width': (self.bounds['east'] - self.bounds['west']) / self.resolution,
                'pixel_height': (self.bounds['north'] - self.bounds['south']) / self.resolution
            }
            
            # Check coverage
            alpha_channel = self.raster_array[:, :, 3]
            coverage_pct = (alpha_channel > 0).sum() / alpha_channel.size * 100
            
            print(f"✓ Fetched soil raster: {self.raster_array.shape}")
            print(f"  Tiles with data: {tiles_with_data}/{n_tiles_x * n_tiles_y}")
            print(f"  Coverage: {coverage_pct:.1f}%")
            
            if coverage_pct < 10:
                print("  ⚠️ Low coverage - area may be outside BGS soil data extent")
            
            # Classify soil types from raster values
            self._classify_soil_types()
            
            return self.raster_array
            
        except Exception as e:
            print(f"✗ BGS WMS failed: {e}")
            print("  Generating synthetic raster...")
            return self._generate_synthetic_raster()
    
    def _generate_synthetic_raster(self):
        """Generate synthetic soil raster for testing."""
        np.random.seed(42)  # Reproducible
        
        # Create smooth soil variation using perlin-like noise
        x = np.linspace(0, 4, self.resolution)
        y = np.linspace(0, 4, self.resolution)
        xx, yy = np.meshgrid(x, y)
        
        # Combine multiple frequencies for realistic variation
        noise = (np.sin(xx * 2) * np.cos(yy * 2) + 
                 np.sin(xx * 4 + 1) * np.cos(yy * 4 + 1) * 0.5 +
                 np.random.random((self.resolution, self.resolution)) * 0.3)
        
        # Normalize to 0-255 range
        noise = ((noise - noise.min()) / (noise.max() - noise.min()) * 255).astype(np.uint8)
        
        # Create RGB image (grayscale for simplicity)
        self.raster_array = np.stack([noise, noise, noise, np.full_like(noise, 255)], axis=-1)
        
        # Create geotransform
        self.transform = {
            'west': self.bounds['west'],
            'north': self.bounds['north'],
            'pixel_width': (self.bounds['east'] - self.bounds['west']) / self.resolution,
            'pixel_height': (self.bounds['north'] - self.bounds['south']) / self.resolution
        }
        
        print(f"✓ Generated synthetic soil raster: {self.raster_array.shape}")
        
        # Classify soil types
        self._classify_soil_types()
        
        return self.raster_array
    
    def _classify_soil_types(self):
        """Classify raster values into soil types, handling transparent pixels."""
        if self.raster_array is None:
            return
        
        # Get grayscale value and alpha channel
        if len(self.raster_array.shape) == 3:
            gray = self.raster_array[:, :, 0].astype(float)
            alpha = self.raster_array[:, :, 3]
        else:
            gray = self.raster_array.astype(float)
            alpha = np.full_like(gray, 255)
        
        # Invert so higher values = more clay
        clay_index = 255 - gray
        
        # Create soil type classification map
        self.soil_type_map = np.zeros_like(clay_index, dtype=np.uint8)
        
        # Classify based on clay index thresholds
        self.soil_type_map[clay_index < 40] = 1    # Sand
        self.soil_type_map[(clay_index >= 40) & (clay_index < 80)] = 2    # Sandy Loam
        self.soil_type_map[(clay_index >= 80) & (clay_index < 120)] = 3   # Loam
        self.soil_type_map[(clay_index >= 120) & (clay_index < 160)] = 4  # Clay Loam
        self.soil_type_map[(clay_index >= 160) & (clay_index < 200)] = 5  # Clay
        self.soil_type_map[clay_index >= 200] = 6  # Heavy Clay
        
        # Mark transparent pixels (no data) as Unknown (code 0)
        self.soil_type_map[alpha == 0] = 0
        
        # Mapping for lookup
        self.soil_type_names = {
            0: 'Unknown',
            1: 'Sand',
            2: 'Sandy Loam', 
            3: 'Loam',
            4: 'Clay Loam',
            5: 'Clay',
            6: 'Heavy Clay'
        }
        
        print(f"  Soil type distribution:")
        unique, counts = np.unique(self.soil_type_map, return_counts=True)
        for u, c in zip(unique, counts):
            pct = c / self.soil_type_map.size * 100
            name = self.soil_type_names.get(u, 'Unknown')
            if pct > 0.1:  # Only show types with >0.1% coverage
                print(f"    {name}: {pct:.1f}%")
    
    def get_soil_at_point(self, lon, lat):
        """
        Get soil type at a specific coordinate.
        
        Returns:
        --------
        tuple: (soil_type_code, soil_type_name, clay_index)
        """
        if self.raster_array is None or self.transform is None:
            return (0, 'Unknown', 0)
        
        # Convert lon/lat to pixel coordinates
        px = int((lon - self.transform['west']) / self.transform['pixel_width'])
        py = int((self.transform['north'] - lat) / self.transform['pixel_height'])
        
        # Clamp to bounds
        px = max(0, min(px, self.resolution - 1))
        py = max(0, min(py, self.resolution - 1))
        
        soil_code = self.soil_type_map[py, px]
        soil_name = self.soil_type_names.get(soil_code, 'Unknown')
        
        # Get clay index value
        if len(self.raster_array.shape) == 3:
            clay_index = 255 - self.raster_array[py, px, 0]
        else:
            clay_index = 255 - self.raster_array[py, px]
        
        return (soil_code, soil_name, clay_index)
    
    def get_raster_for_folium(self):
        """
        Get raster data formatted for folium ImageOverlay.
        
        Returns:
        --------
        tuple: (image_bounds, colored_array)
        """
        if self.soil_type_map is None:
            return None, None
        
        # Create colored version of soil map
        colors = {
            0: [200, 200, 200, 100],  # Unknown - light gray, semi-transparent
            1: [255, 255, 200, 180],  # Sand - pale yellow
            2: [255, 230, 150, 180],  # Sandy Loam - light orange
            3: [200, 200, 150, 180],  # Loam - tan
            4: [180, 150, 100, 180],  # Clay Loam - brown
            5: [150, 100, 80, 180],   # Clay - dark brown
            6: [100, 60, 40, 180]     # Heavy Clay - dark brown/red
        }
        
        colored = np.zeros((self.resolution, self.resolution, 4), dtype=np.uint8)
        for code, color in colors.items():
            mask = self.soil_type_map == code
            colored[mask] = color
        
        bounds = [[self.bounds['south'], self.bounds['west']], 
                  [self.bounds['north'], self.bounds['east']]]
        
        return bounds, colored

# Initialize soil raster handler
print("=" * 60)
print("INITIALIZING SOIL RASTER DATA")
print("=" * 60)

USE_REAL_BGS_DATA = True  # Set True for real BGS WMS data

soil_handler = SoilRasterHandler(BRISTOL_BOUNDS, resolution=512)
soil_raster = soil_handler.fetch_wms_raster(use_real_data=USE_REAL_BGS_DATA)

print(f"\n✓ Soil raster ready for building analysis")

INITIALIZING SOIL RASTER DATA
Fetching soil raster from BGS WMS (tiled mode)...
  Layer: Parent.Material.Soil.texture
  Grid: 14×9 tiles (126 total requests)
  Tile size: ~1.6km × 1.5km
  Layer: Parent.Material.Soil.texture
  Grid: 14×9 tiles (126 total requests)
  Tile size: ~1.6km × 1.5km
  Progress: 100% (126 tiles with data)
✓ Fetched soil raster: (512, 512, 4)
  Tiles with data: 126/126
  Coverage: 96.9%
  Soil type distribution:
    Unknown: 3.1%
    Sand: 35.5%
    Sandy Loam: 61.1%
    Loam: 0.3%

✓ Soil raster ready for building analysis
  Progress: 100% (126 tiles with data)
✓ Fetched soil raster: (512, 512, 4)
  Tiles with data: 126/126
  Coverage: 96.9%
  Soil type distribution:
    Unknown: 3.1%
    Sand: 35.5%
    Sandy Loam: 61.1%
    Loam: 0.3%

✓ Soil raster ready for building analysis


## 5. Risk Scoring Functions

This section implements the building-level subsidence risk scoring system:

1. **Soil Scoring** - Samples soil type at building centroid and corners
2. **Tree Scoring** - Uses R-tree spatial index for fast proximity queries
3. **Combined Scoring** - Weighted combination with configurable thresholds

In [80]:
# ============================================
# SOIL RISK SCORING FUNCTIONS
# ============================================
# Note: SOIL_RISK_SCORES and SOIL_CODE_SCORES are defined in the 
# Risk Scoring Configuration cell above

def get_soil_risk_score(soil_code):
    """
    Get the subsidence risk score for a soil type.
    
    Parameters:
    -----------
    soil_code : int
        Soil type code from raster classification
    
    Returns:
    --------
    float
        Risk score (0-10)
    """
    return SOIL_CODE_SCORES.get(soil_code, 5.0)

def get_building_soil_score(building_geom, soil_handler):
    """
    Calculate soil risk score for a building.
    
    Samples soil at building centroid and optionally at corners.
    
    Parameters:
    -----------
    building_geom : shapely.geometry
        Building polygon geometry
    soil_handler : SoilRasterHandler
        Soil data handler
    
    Returns:
    --------
    dict
        Soil analysis results including score, type, and details
    """
    # Get building centroid
    centroid = building_geom.centroid
    
    # Sample soil at centroid
    soil_code, soil_name, clay_index = soil_handler.get_soil_at_point(centroid.x, centroid.y)
    
    # Get base score
    soil_score = get_soil_risk_score(soil_code)
    
    # Optionally sample at building corners and average
    if hasattr(building_geom, 'exterior'):
        coords = list(building_geom.exterior.coords)
        corner_scores = []
        for x, y in coords[:4]:  # Sample up to 4 corners
            code, _, _ = soil_handler.get_soil_at_point(x, y)
            corner_scores.append(get_soil_risk_score(code))
        
        if corner_scores:
            # Weighted average: 60% centroid, 40% corners
            avg_corner = np.mean(corner_scores)
            soil_score = soil_score * 0.6 + avg_corner * 0.4
    
    return {
        'soil_score': soil_score,
        'soil_type': soil_name,
        'soil_code': soil_code,
        'clay_index': clay_index
    }

print("✓ Soil risk scoring functions loaded (using config from above)")

✓ Soil risk scoring functions loaded (using config from above)


In [100]:
# ============================================
# TREE PROXIMITY RISK SCORING FUNCTIONS (OPTIMIZED)
# ============================================
# Note: TREE_CONFIG and SPECIES_RISK_FACTORS are defined in the 
# Risk Scoring Configuration cell above

# Build spatial index for fast tree lookups
_tree_sindex = None
_tree_data_indexed = None

def build_tree_spatial_index(tree_data):
    """Build a spatial index for fast tree proximity lookups."""
    global _tree_sindex, _tree_data_indexed
    
    if tree_data is None or tree_data.empty:
        _tree_sindex = None
        _tree_data_indexed = None
        return
    
    print(f"  Building spatial index for {len(tree_data):,} trees...")
    _tree_data_indexed = tree_data.copy()
    _tree_sindex = tree_data.sindex  # R-tree spatial index
    print(f"  ✓ Spatial index built")

def get_species_factor(species_name):
    """
    Get the risk factor for a tree species.
    """
    if pd.isna(species_name) or not species_name:
        return SPECIES_RISK_FACTORS['default']
    
    species_lower = str(species_name).lower()
    
    for key, factor in SPECIES_RISK_FACTORS.items():
        if key in species_lower:
            return factor
    
    return SPECIES_RISK_FACTORS['default']

def calculate_tree_influence(tree_row, building_centroid, crs='EPSG:4326'):
    """
    Calculate the influence of a single tree on a building.
    """
    tree_point = tree_row.geometry
    
    # Calculate distance (in degrees, convert to approximate meters)
    dist_deg = tree_point.distance(building_centroid)
    dist_m = dist_deg * 111000  # Rough conversion at UK latitude
    
    # Get crown width (proxy for root spread) - convert to float if string
    crown_width = tree_row.get('CROWN_WIDTH', TREE_CONFIG['default_crown_width_m'])
    try:
        crown_width = float(crown_width) if crown_width is not None else TREE_CONFIG['default_crown_width_m']
    except (ValueError, TypeError):
        crown_width = TREE_CONFIG['default_crown_width_m']
    if pd.isna(crown_width) or crown_width <= 0:
        crown_width = TREE_CONFIG['default_crown_width_m']
    
    # Estimate root spread zone
    root_spread = crown_width * TREE_CONFIG['root_spread_multiplier']
    
    # Get species factor
    species_name = tree_row.get('COMMON_NAME', tree_row.get('LATIN_NAME', ''))
    species_factor = get_species_factor(species_name)
    
    # Get DBH - convert to float if string
    dbh = tree_row.get('DBH', 0)
    try:
        dbh = float(dbh) if dbh is not None else 0
    except (ValueError, TypeError):
        dbh = 0
    if pd.isna(dbh):
        dbh = 0
    dbh_factor = 1.0 + (dbh / 100) if dbh > 0 else 1.0
    
    # Calculate influence - designed to produce meaningful 0-10 scores
    if dist_m <= 0:
        dist_m = 0.1
    
    max_dist = TREE_CONFIG['max_influence_distance_m']
    
    if dist_m > max_dist:
        influence = 0
    elif dist_m <= root_spread:
        # Within root zone - HIGH influence
        # Base score of 3.0 for being in root zone, scaling up for closer trees
        proximity_factor = 1 - (dist_m / root_spread) ** 0.5  # 0 to 1, higher when closer
        influence = TREE_CONFIG['base_tree_score'] * (0.5 + 0.5 * proximity_factor) * species_factor * dbh_factor
        
        # Bonus for very close trees (< 5m)
        if dist_m < 5:
            influence += TREE_CONFIG['close_tree_bonus'] * (1 - dist_m / 5)
    else:
        # Outside root zone but within influence distance - decreasing influence
        # Linear decay from root_spread to max_influence_distance
        remaining_dist = max_dist - root_spread
        dist_beyond_root = dist_m - root_spread
        decay_factor = 1 - (dist_beyond_root / remaining_dist) ** TREE_CONFIG['distance_decay_power']
        influence = 1.5 * decay_factor * species_factor * dbh_factor
    
    return {
        'distance_m': dist_m,
        'crown_width': crown_width,
        'root_spread': root_spread,
        'species_factor': species_factor,
        'influence': influence,
        'within_root_zone': dist_m <= root_spread
    }

def get_building_tree_score(building_geom, tree_data, max_trees=None):
    """
    Calculate total tree proximity risk score for a building.
    Uses spatial index for fast lookups.
    """
    global _tree_sindex, _tree_data_indexed
    
    if tree_data is None or tree_data.empty:
        return {
            'tree_score': 0,
            'nearby_trees': 0,
            'trees_in_root_zone': 0,
            'closest_tree_m': None,
            'tree_details': []
        }
    
    max_trees = max_trees or TREE_CONFIG['max_trees_to_consider']
    
    # Get building centroid
    centroid = building_geom.centroid
    
    # Find nearby trees using spatial index (MUCH faster)
    max_dist_deg = TREE_CONFIG['max_influence_distance_m'] / 111000
    
    # Create search bounding box
    search_bbox = (
        centroid.x - max_dist_deg,
        centroid.y - max_dist_deg,
        centroid.x + max_dist_deg,
        centroid.y + max_dist_deg
    )
    
    # Use spatial index if available
    if _tree_sindex is not None and _tree_data_indexed is not None:
        # Query spatial index - returns indices of potentially matching trees
        possible_matches_idx = list(_tree_sindex.intersection(search_bbox))
        if not possible_matches_idx:
            return {
                'tree_score': 0,
                'nearby_trees': 0,
                'trees_in_root_zone': 0,
                'closest_tree_m': None,
                'tree_details': []
            }
        nearby_trees = _tree_data_indexed.iloc[possible_matches_idx].head(max_trees)
    else:
        # Fallback to slower method
        nearby_mask = (
            (tree_data.geometry.x >= search_bbox[0]) & 
            (tree_data.geometry.x <= search_bbox[2]) &
            (tree_data.geometry.y >= search_bbox[1]) & 
            (tree_data.geometry.y <= search_bbox[3])
        )
        nearby_trees = tree_data[nearby_mask].head(max_trees)
    
    if nearby_trees.empty:
        return {
            'tree_score': 0,
            'nearby_trees': 0,
            'trees_in_root_zone': 0,
            'closest_tree_m': None,
            'tree_details': []
        }
    
    # Calculate influence from each tree
    total_influence = 0
    trees_in_root_zone = 0
    closest_distance = float('inf')
    tree_details = []
    
    for idx, tree in nearby_trees.iterrows():
        result = calculate_tree_influence(tree, centroid)
        
        total_influence += result['influence']
        
        if result['within_root_zone']:
            trees_in_root_zone += 1
        
        if result['distance_m'] < closest_distance:
            closest_distance = result['distance_m']
        
        if result['influence'] > 0.1:
            tree_details.append({
                'tree_id': tree.get('ASSET_ID', idx),
                'species': tree.get('COMMON_NAME', 'Unknown'),
                'distance_m': result['distance_m'],
                'influence': result['influence']
            })
    
    # Cap the tree score - with new scoring, total_influence is already on ~0-10 scale
    tree_score = min(TREE_CONFIG['score_cap'], total_influence)
    
    return {
        'tree_score': tree_score,
        'nearby_trees': len(nearby_trees),
        'trees_in_root_zone': trees_in_root_zone,
        'closest_tree_m': closest_distance if closest_distance < float('inf') else None,
        'total_influence': total_influence,
        'tree_details': sorted(tree_details, key=lambda x: x['influence'], reverse=True)[:5]
    }

# Build the spatial index now if tree data is available
if 'tree_data' in dir() and tree_data is not None and not tree_data.empty:
    build_tree_spatial_index(tree_data)

print("✓ Tree proximity risk scoring functions loaded (with spatial index optimization)")


  Building spatial index for 55,235 trees...
  ✓ Spatial index built
✓ Tree proximity risk scoring functions loaded (with spatial index optimization)


In [101]:
# ============================================
# COMBINED BUILDING RISK SCORING FUNCTIONS
# ============================================
# Note: RISK_WEIGHTS and RISK_THRESHOLDS are defined in the 
# Risk Scoring Configuration cell above

def calculate_building_risk(building_geom, soil_handler, tree_data):
    """
    Calculate combined subsidence risk score for a building.
    
    Parameters:
    -----------
    building_geom : shapely.geometry
        Building polygon geometry
    soil_handler : SoilRasterHandler
        Soil raster data handler
    tree_data : GeoDataFrame
        Tree dataset
    
    Returns:
    --------
    dict
        Complete risk assessment for the building
    """
    # Get soil risk
    soil_result = get_building_soil_score(building_geom, soil_handler)
    
    # Get tree risk
    tree_result = get_building_tree_score(building_geom, tree_data)
    
    # Calculate combined score (weighted average)
    combined_score = (
        soil_result['soil_score'] * RISK_WEIGHTS['soil'] +
        tree_result['tree_score'] * RISK_WEIGHTS['tree']
    )
    
    # Determine risk category using thresholds from config
    if combined_score >= RISK_THRESHOLDS['high']:
        risk_category = 'High'
    elif combined_score >= RISK_THRESHOLDS['medium']:
        risk_category = 'Medium'
    else:
        risk_category = 'Low'
    
    return {
        'combined_score': combined_score,
        'risk_category': risk_category,
        'soil_score': soil_result['soil_score'],
        'soil_type': soil_result['soil_type'],
        'tree_score': tree_result['tree_score'],
        'nearby_trees': tree_result['nearby_trees'],
        'trees_in_root_zone': tree_result['trees_in_root_zone'],
        'closest_tree_m': tree_result['closest_tree_m'],
        'top_tree_contributors': tree_result['tree_details'][:3] if tree_result['tree_details'] else []
    }

def score_all_buildings(building_data, soil_handler, tree_data, progress_interval=50):
    """
    Calculate risk scores for all buildings.
    
    Parameters:
    -----------
    building_data : GeoDataFrame
        Building footprints
    soil_handler : SoilRasterHandler
        Soil raster data handler
    tree_data : GeoDataFrame
        Tree dataset
    progress_interval : int
        Print progress every N buildings
    
    Returns:
    --------
    GeoDataFrame
        Buildings with risk scores added
    """
    print("=" * 60)
    print("CALCULATING BUILDING RISK SCORES")
    print("=" * 60)
    print(f"\nProcessing {len(building_data)} buildings...")
    print(f"Risk weights: Soil={RISK_WEIGHTS['soil']:.0%}, Trees={RISK_WEIGHTS['tree']:.0%}")
    print(f"Risk thresholds: High≥{RISK_THRESHOLDS['high']}, Medium≥{RISK_THRESHOLDS['medium']}")
    
    # Prepare results columns
    results = []
    
    for idx, (i, building) in enumerate(building_data.iterrows()):
        if idx > 0 and idx % progress_interval == 0:
            print(f"  Processed {idx}/{len(building_data)} buildings...")
        
        # Calculate risk for this building
        risk = calculate_building_risk(building.geometry, soil_handler, tree_data)
        results.append(risk)
    
    # Add results to building data
    scored_buildings = building_data.copy()
    
    scored_buildings['risk_score'] = [r['combined_score'] for r in results]
    scored_buildings['risk_category'] = [r['risk_category'] for r in results]
    scored_buildings['soil_score'] = [r['soil_score'] for r in results]
    scored_buildings['soil_type'] = [r['soil_type'] for r in results]
    scored_buildings['tree_score'] = [r['tree_score'] for r in results]
    scored_buildings['nearby_trees'] = [r['nearby_trees'] for r in results]
    scored_buildings['trees_in_root_zone'] = [r['trees_in_root_zone'] for r in results]
    scored_buildings['closest_tree_m'] = [r['closest_tree_m'] for r in results]
    
    print(f"\n✓ Completed scoring for {len(scored_buildings)} buildings")
    
    # Summary statistics
    print(f"\n{'=' * 60}")
    print("RISK SUMMARY")
    print(f"{'=' * 60}")
    
    print(f"\nRisk Category Distribution:")
    for cat in ['High', 'Medium', 'Low']:
        count = len(scored_buildings[scored_buildings['risk_category'] == cat])
        pct = count / len(scored_buildings) * 100
        print(f"  {cat:8s}: {count:4d} buildings ({pct:5.1f}%)")
    
    print(f"\nScore Statistics:")
    print(f"  Combined: mean={scored_buildings['risk_score'].mean():.2f}, "
          f"min={scored_buildings['risk_score'].min():.2f}, "
          f"max={scored_buildings['risk_score'].max():.2f}")
    print(f"  Soil:     mean={scored_buildings['soil_score'].mean():.2f}, "
          f"min={scored_buildings['soil_score'].min():.2f}, "
          f"max={scored_buildings['soil_score'].max():.2f}")
    print(f"  Tree:     mean={scored_buildings['tree_score'].mean():.2f}, "
          f"min={scored_buildings['tree_score'].min():.2f}, "
          f"max={scored_buildings['tree_score'].max():.2f}")
    
    print(f"\nSoil Type Distribution:")
    print(scored_buildings['soil_type'].value_counts())
    
    return scored_buildings

print("✓ Combined risk scoring functions loaded (using config from above)")

✓ Combined risk scoring functions loaded (using config from above)


In [102]:
# ============================================
# RUN BUILDING RISK ANALYSIS
# ============================================

# Load building data if not already loaded
if 'building_data' not in dir() or building_data.empty:
    try:
        building_data = gpd.read_file('bristol_buildings_tiny.geojson')
        print(f"Loaded {len(building_data)} buildings from file")
    except:
        print("❌ Building data not found. Run the building collection cell first.")
        building_data = gpd.GeoDataFrame()

# Load tree data if not already loaded
if 'tree_data' not in dir() or tree_data.empty:
    print("⚠️ Tree data not loaded. Run the tree collection cell first for full analysis.")
    tree_data = gpd.GeoDataFrame()

# Filter buildings to be within soil raster bounds
if not building_data.empty and 'soil_handler' in dir():
    original_count = len(building_data)
    
    # Get soil bounds
    soil_bounds = soil_handler.bounds
    
    # Filter buildings whose centroid is within soil bounds
    building_centroids = building_data.geometry.centroid
    within_bounds_mask = (
        (building_centroids.x >= soil_bounds['west']) &
        (building_centroids.x <= soil_bounds['east']) &
        (building_centroids.y >= soil_bounds['south']) &
        (building_centroids.y <= soil_bounds['north'])
    )
    
    building_data_filtered = building_data[within_bounds_mask].copy()
    
    filtered_count = len(building_data_filtered)
    removed_count = original_count - filtered_count
    
    print(f"📍 Filtering buildings to soil raster bounds:")
    print(f"   Original: {original_count:,} buildings")
    print(f"   Within bounds: {filtered_count:,} buildings")
    print(f"   Removed: {removed_count:,} buildings ({removed_count/original_count*100:.1f}%)")
    
    # Use filtered buildings for scoring
    buildings_to_score = building_data_filtered
else:
    buildings_to_score = building_data

# Score all buildings
if not buildings_to_score.empty:
    scored_buildings = score_all_buildings(
        buildings_to_score, 
        soil_handler, 
        tree_data,
        progress_interval=1000
    )
    
    # Save scored buildings
    scored_buildings.to_file('bristol_buildings_scored.geojson', driver='GeoJSON')
    print(f"\n✓ Saved scored buildings to: bristol_buildings_scored.geojson")
else:
    print("❌ No building data available for scoring")

📍 Filtering buildings to soil raster bounds:
   Original: 227,578 buildings
   Within bounds: 227,361 buildings
   Removed: 217 buildings (0.1%)
CALCULATING BUILDING RISK SCORES

Processing 227361 buildings...
Risk weights: Soil=40%, Trees=60%
Risk thresholds: High≥7.0, Medium≥4.0
  Processed 1000/227361 buildings...
  Processed 2000/227361 buildings...
  Processed 2000/227361 buildings...
  Processed 3000/227361 buildings...
  Processed 3000/227361 buildings...
  Processed 4000/227361 buildings...
  Processed 4000/227361 buildings...
  Processed 5000/227361 buildings...
  Processed 5000/227361 buildings...
  Processed 6000/227361 buildings...
  Processed 6000/227361 buildings...
  Processed 7000/227361 buildings...
  Processed 7000/227361 buildings...
  Processed 8000/227361 buildings...
  Processed 8000/227361 buildings...
  Processed 9000/227361 buildings...
  Processed 9000/227361 buildings...
  Processed 10000/227361 buildings...
  Processed 10000/227361 buildings...
  Processed 1

## 6. Visualization & Export

### 6.1 Interactive Multi-Layer Map

Creates an interactive Folium map with toggleable layers:
- **🌍 Soil Raster** - BGS soil texture overlay with transparency
- **🔴 High Risk Buildings** - Score ≥ 7.0 (red)
- **🟡 Medium Risk Buildings** - Score 4.0-7.0 (orange)
- **🟢 Low Risk Buildings** - Score < 4.0 (green)
- **🌳 Trees** - With crown width and species info (off by default)

**Note:** For performance, only the top N highest-risk buildings are displayed (configurable via `MAX_BUILDINGS_DISPLAY`).

In [103]:
# ============================================
# MULTI-LAYER INTERACTIVE MAP
# ============================================

import base64
from io import BytesIO
from PIL import Image

# Configuration for map display
MAX_BUILDINGS_DISPLAY = 10000  # Limit buildings to prevent browser crashes

def create_risk_map(scored_buildings, tree_data, soil_handler, max_buildings=MAX_BUILDINGS_DISPLAY, show_all_trees=True):
    """
    Create interactive map with toggleable layers.
    
    Parameters:
    -----------
    scored_buildings : GeoDataFrame
        Buildings with risk scores
    tree_data : GeoDataFrame
        Tree dataset
    soil_handler : SoilRasterHandler
        Soil raster data
    max_buildings : int
        Maximum number of buildings to display (shows highest risk first)
    show_all_trees : bool
        If False, only show trees within influence distance of buildings
    
    Returns:
    --------
    folium.Map
        Interactive map with all layers
    """
    print("Creating multi-layer risk map...")
    
    # FIRST: Filter buildings to only those within soil bounds
    soil_bounds = soil_handler.bounds
    building_centroids = scored_buildings.geometry.centroid
    within_soil_mask = (
        (building_centroids.x >= soil_bounds['west']) &
        (building_centroids.x <= soil_bounds['east']) &
        (building_centroids.y >= soil_bounds['south']) &
        (building_centroids.y <= soil_bounds['north'])
    )
    
    buildings_in_soil_area = scored_buildings[within_soil_mask].copy()
    filtered_bounds = len(scored_buildings) - len(buildings_in_soil_area)
    
    if filtered_bounds > 0:
        print(f"  📍 Filtered {filtered_bounds:,} buildings outside soil bounds")
    
    # Note: Buildings with Unknown soil type have soil_score=0, so they won't
    # appear in top N highest risk unless they have high tree proximity scores
    
    total_in_area = len(buildings_in_soil_area)
    
    # THEN: Filter to top N highest risk buildings
    if total_in_area > max_buildings:
        print(f"  ⚠️ {total_in_area:,} buildings in soil area - displaying top {max_buildings:,} highest risk")
        buildings_to_show = buildings_in_soil_area.nlargest(max_buildings, 'risk_score').copy()
    else:
        buildings_to_show = buildings_in_soil_area.copy()
        print(f"  Displaying all {len(buildings_to_show):,} buildings")
    
    # Get map center from soil bounds (ensures alignment)
    center_lat = (soil_bounds['north'] + soil_bounds['south']) / 2
    center_lon = (soil_bounds['east'] + soil_bounds['west']) / 2
    
    # Create base map
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=13,
        tiles='cartodbpositron',
        control_scale=True
    )
    
    # ==========================================
    # LAYER 1: Soil Raster Overlay
    # ==========================================
    print("  Adding soil raster layer...")
    
    bounds, soil_colored = soil_handler.get_raster_for_folium()
    
    if soil_colored is not None:
        img = Image.fromarray(soil_colored)
        buffer = BytesIO()
        img.save(buffer, format='PNG')
        img_str = base64.b64encode(buffer.getvalue()).decode()
        
        soil_layer = folium.FeatureGroup(name='🌍 Soil Type Raster', show=True)
        
        folium.raster_layers.ImageOverlay(
            image=f'data:image/png;base64,{img_str}',
            bounds=bounds,
            opacity=0.5,
            name='Soil'
        ).add_to(soil_layer)
        
        soil_layer.add_to(m)
    
    # ==========================================
    # LAYER 2: Buildings by Risk Score
    # ==========================================
    print(f"  Adding {len(buildings_to_show):,} building layers...")
    
    def get_risk_color(score):
        if score >= 7:
            return '#e74c3c'  # Red - High risk
        elif score >= 4:
            return '#f39c12'  # Orange - Medium risk
        else:
            return '#27ae60'  # Green - Low risk
    
    def get_risk_fill_opacity(score):
        return 0.4 + (score / 10) * 0.4
    
    high_risk_layer = folium.FeatureGroup(name='🔴 High Risk Buildings', show=True)
    medium_risk_layer = folium.FeatureGroup(name='🟡 Medium Risk Buildings', show=True)
    low_risk_layer = folium.FeatureGroup(name='🟢 Low Risk Buildings', show=True)
    
    for idx, building in buildings_to_show.iterrows():
        score = building.get('risk_score', 0)
        category = building.get('risk_category', 'Unknown')
        
        closest_tree = building.get('closest_tree_m')
        closest_tree_str = f"{closest_tree:.1f}m" if closest_tree is not None and not pd.isna(closest_tree) else "N/A"
        
        popup_html = f"""
        <div style="width: 250px; font-family: Arial, sans-serif;">
            <h4 style="margin: 0 0 10px 0; color: {get_risk_color(score)};">
                {category} Risk
            </h4>
            <table style="width: 100%; font-size: 12px;">
                <tr><td><b>Combined Score:</b></td><td>{score:.2f}/10</td></tr>
                <tr><td><b>Soil Score:</b></td><td>{building.get('soil_score', 0):.2f}/10</td></tr>
                <tr><td><b>Tree Score:</b></td><td>{building.get('tree_score', 0):.2f}/10</td></tr>
                <tr><td><b>Soil Type:</b></td><td>{building.get('soil_type', 'Unknown')}</td></tr>
                <tr><td><b>Nearby Trees:</b></td><td>{building.get('nearby_trees', 0)}</td></tr>
                <tr><td><b>Trees in Root Zone:</b></td><td>{building.get('trees_in_root_zone', 0)}</td></tr>
                <tr><td><b>Closest Tree:</b></td><td>{closest_tree_str}</td></tr>
            </table>
            <p style="font-size: 10px; color: #666; margin-top: 10px;">
                Building: {building.get('building', 'Unknown type')}<br>
                OSM ID: {building.get('osm_id', 'N/A')}
            </p>
        </div>
        """
        
        style = {
            'fillColor': get_risk_color(score),
            'color': '#333',
            'weight': 1,
            'fillOpacity': get_risk_fill_opacity(score)
        }
        
        feature = folium.GeoJson(
            building.geometry.__geo_interface__,
            style_function=lambda x, s=style: s,
            popup=folium.Popup(popup_html, max_width=300),
            tooltip=f"Risk: {score:.1f}/10 ({category})"
        )
        
        if category == 'High':
            feature.add_to(high_risk_layer)
        elif category == 'Medium':
            feature.add_to(medium_risk_layer)
        else:
            feature.add_to(low_risk_layer)
    
    low_risk_layer.add_to(m)
    medium_risk_layer.add_to(m)
    high_risk_layer.add_to(m)
    
    # ==========================================
    # LAYER 3: Trees (near displayed buildings only)
    # ==========================================
    if tree_data is not None and not tree_data.empty:
        # Filter trees to soil bounds
        trees_in_area = tree_data[
            (tree_data.geometry.x >= soil_bounds['west']) &
            (tree_data.geometry.x <= soil_bounds['east']) &
            (tree_data.geometry.y >= soil_bounds['south']) &
            (tree_data.geometry.y <= soil_bounds['north'])
        ]
        
        print(f"  Adding tree layer ({len(trees_in_area):,} trees in soil area)...")
        
        tree_layer = folium.FeatureGroup(name='🌳 Trees', show=False)
        
        max_trees_display = 2000
        if len(trees_in_area) > max_trees_display:
            trees_to_show = trees_in_area.sample(max_trees_display)
            print(f"    (Showing {max_trees_display:,} of {len(trees_in_area):,} trees)")
        else:
            trees_to_show = trees_in_area
        
        for idx, tree in trees_to_show.iterrows():
            crown_width = tree.get('CROWN_WIDTH', 5)
            try:
                crown_width = float(crown_width) if crown_width is not None else 5
            except (ValueError, TypeError):
                crown_width = 5
            if pd.isna(crown_width) or crown_width <= 0:
                crown_width = 5
            
            species = tree.get('COMMON_NAME', 'Unknown')
            if pd.isna(species):
                species = tree.get('LATIN_NAME', 'Unknown')
            
            radius = max(3, min(12, crown_width / 2))
            
            tree_popup = f"""
            <div style="width: 200px;">
                <b>{species}</b><br>
                Crown Width: {crown_width:.1f}m<br>
                Crown Area: {tree.get('CROWN_AREA', 'N/A')}<br>
                DBH: {tree.get('DBH', 'N/A')}<br>
                Height: {tree.get('CROWN_HEIGHT', 'N/A')}m
            </div>
            """
            
            folium.CircleMarker(
                location=[tree.geometry.y, tree.geometry.x],
                radius=radius,
                color='darkgreen',
                fill=True,
                fillColor='green',
                fillOpacity=0.6,
                weight=1,
                popup=folium.Popup(tree_popup, max_width=250),
                tooltip=f"{species}"
            ).add_to(tree_layer)
        
        tree_layer.add_to(m)
    
    # ==========================================
    # LAYER 4: Legend
    # ==========================================
    legend_html = f"""
    <div style="position: fixed; 
                bottom: 50px; left: 50px; 
                width: 220px; 
                background-color: white; 
                border: 2px solid #333; 
                border-radius: 5px;
                padding: 10px;
                font-family: Arial, sans-serif;
                font-size: 12px;
                z-index: 9999;">
        <h4 style="margin: 0 0 10px 0;">Subsidence Risk</h4>
        <div style="margin: 5px 0;">
            <span style="background: #e74c3c; width: 20px; height: 12px; display: inline-block; margin-right: 5px;"></span>
            High (≥7)
        </div>
        <div style="margin: 5px 0;">
            <span style="background: #f39c12; width: 20px; height: 12px; display: inline-block; margin-right: 5px;"></span>
            Medium (4-7)
        </div>
        <div style="margin: 5px 0;">
            <span style="background: #27ae60; width: 20px; height: 12px; display: inline-block; margin-right: 5px;"></span>
            Low (<4)
        </div>
        <hr style="margin: 10px 0;">
        <h4 style="margin: 0 0 10px 0;">Soil Types</h4>
        <div style="margin: 5px 0;">
            <span style="background: rgb(100,60,40); width: 20px; height: 12px; display: inline-block; margin-right: 5px;"></span>
            Heavy Clay
        </div>
        <div style="margin: 5px 0;">
            <span style="background: rgb(180,150,100); width: 20px; height: 12px; display: inline-block; margin-right: 5px;"></span>
            Clay Loam
        </div>
        <div style="margin: 5px 0;">
            <span style="background: rgb(255,255,200); width: 20px; height: 12px; display: inline-block; margin-right: 5px;"></span>
            Sand
        </div>
        <hr style="margin: 10px 0;">
        <div style="font-size: 10px; color: #666;">
            Showing top {max_buildings:,} highest<br>
            risk of {total_in_area:,} buildings<br>
            within soil data area
        </div>
    </div>
    """
    m.get_root().html.add_child(folium.Element(legend_html))
    
    # ==========================================
    # Add Layer Control
    # ==========================================
    folium.LayerControl(collapsed=False).add_to(m)
    
    print("✓ Map created successfully")
    
    return m

# Create the map
if 'scored_buildings' in dir() and not scored_buildings.empty:
    risk_map = create_risk_map(
        scored_buildings,
        tree_data if 'tree_data' in dir() else gpd.GeoDataFrame(),
        soil_handler,
        max_buildings=MAX_BUILDINGS_DISPLAY
    )
    
    # Save map
    risk_map.save('subsidence_risk_buildings.html')
    print(f"\n✓ Map saved to: subsidence_risk_buildings.html")
    
    # Display in notebook
    risk_map
else:
    print("❌ Run the building scoring cell first to create scored_buildings")

Creating multi-layer risk map...
  ⚠️ 227,361 buildings in soil area - displaying top 10,000 highest risk
  Adding soil raster layer...
  Adding 10,000 building layers...
  ⚠️ 227,361 buildings in soil area - displaying top 10,000 highest risk
  Adding soil raster layer...
  Adding 10,000 building layers...
  Adding tree layer (55,235 trees in soil area)...
    (Showing 2,000 of 55,235 trees)
  Adding tree layer (55,235 trees in soil area)...
    (Showing 2,000 of 55,235 trees)
✓ Map created successfully
✓ Map created successfully

✓ Map saved to: subsidence_risk_buildings.html

✓ Map saved to: subsidence_risk_buildings.html


In [68]:
# ============================================
# DETAILED BUILDING ANALYSIS
# ============================================

def analyze_high_risk_buildings(scored_buildings, top_n=10):
    """
    Detailed analysis of highest risk buildings.
    
    Parameters:
    -----------
    scored_buildings : GeoDataFrame
        Buildings with risk scores
    top_n : int
        Number of top buildings to analyze
    """
    print("=" * 60)
    print(f"TOP {top_n} HIGHEST RISK BUILDINGS")
    print("=" * 60)
    
    # Sort by risk score
    high_risk = scored_buildings.nlargest(top_n, 'risk_score')
    
    for i, (idx, building) in enumerate(high_risk.iterrows(), 1):
        print(f"\n{'─' * 50}")
        print(f"#{i} - OSM ID: {building.get('osm_id', 'N/A')}")
        print(f"{'─' * 50}")
        print(f"  Risk Score:     {building['risk_score']:.2f}/10 ({building['risk_category']})")
        print(f"  Soil Score:     {building['soil_score']:.2f}/10")
        print(f"  Tree Score:     {building['tree_score']:.2f}/10")
        print(f"  Soil Type:      {building['soil_type']}")
        print(f"  Building Type:  {building.get('building', 'Unknown')}")
        print(f"  Nearby Trees:   {building['nearby_trees']}")
        print(f"  In Root Zone:   {building['trees_in_root_zone']}")
        if building['closest_tree_m']:
            print(f"  Closest Tree:   {building['closest_tree_m']:.1f}m")
        
        # Get centroid for reference
        centroid = building.geometry.centroid
        print(f"  Location:       ({centroid.y:.6f}, {centroid.x:.6f})")
    
    return high_risk

# Show high risk buildings analysis
if 'scored_buildings' in dir() and not scored_buildings.empty:
    high_risk_buildings = analyze_high_risk_buildings(scored_buildings, top_n=10)
else:
    print("Run the scoring cell first to analyze buildings")

TOP 10 HIGHEST RISK BUILDINGS

──────────────────────────────────────────────────
#1 - OSM ID: 23695879
──────────────────────────────────────────────────
  Risk Score:     3.00/10 (Low)
  Soil Score:     5.00/10
  Tree Score:     0.00/10
  Soil Type:      Unknown
  Building Type:  school
  Nearby Trees:   0
  In Root Zone:   0
  Closest Tree:   nanm
  Location:       (51.499139, -2.501894)

──────────────────────────────────────────────────
#2 - OSM ID: 23951773
──────────────────────────────────────────────────
  Risk Score:     3.00/10 (Low)
  Soil Score:     5.00/10
  Tree Score:     0.00/10
  Soil Type:      Unknown
  Building Type:  hangar
  Nearby Trees:   0
  In Root Zone:   0
  Closest Tree:   nanm
  Location:       (51.481431, -2.501232)

──────────────────────────────────────────────────
#3 - OSM ID: 42462950
──────────────────────────────────────────────────
  Risk Score:     3.00/10 (Low)
  Soil Score:     5.00/10
  Tree Score:     0.00/10
  Soil Type:      Unknown
  Build

In [ ]:
# ============================================
# EXPORT RESULTS
# ============================================

def export_results(scored_buildings, output_prefix='bristol_subsidence'):
    """
    Export scored buildings in multiple formats.
    
    Parameters:
    -----------
    scored_buildings : GeoDataFrame
        Buildings with risk scores
    output_prefix : str
        Prefix for output filenames
    """
    print("=" * 60)
    print("EXPORTING RESULTS")
    print("=" * 60)
    
    # 1. GeoJSON (for web mapping)
    geojson_file = f'{output_prefix}_buildings.geojson'
    scored_buildings.to_file(geojson_file, driver='GeoJSON')
    print(f"✓ GeoJSON: {geojson_file}")
    
    # 2. Shapefile (for GIS software)
    shp_file = f'{output_prefix}_buildings.shp'
    # Shapefiles have column name limits, so truncate
    export_gdf = scored_buildings.copy()
    export_gdf.columns = [c[:10] for c in export_gdf.columns]
    export_gdf.to_file(shp_file)
    print(f"✓ Shapefile: {shp_file}")
    
    # 3. CSV summary (for spreadsheets)
    csv_file = f'{output_prefix}_summary.csv'
    csv_cols = ['osm_id', 'building', 'risk_score', 'risk_category', 
                'soil_score', 'soil_type', 'tree_score', 
                'nearby_trees', 'trees_in_root_zone', 'closest_tree_m']
    
    # Add centroid coordinates
    export_df = scored_buildings[csv_cols].copy()
    export_df['centroid_lat'] = scored_buildings.geometry.centroid.y
    export_df['centroid_lon'] = scored_buildings.geometry.centroid.x
    export_df.to_csv(csv_file, index=False)
    print(f"✓ CSV: {csv_file}")
    
    # 4. Summary statistics
    stats_file = f'{output_prefix}_statistics.txt'
    with open(stats_file, 'w') as f:
        f.write("SUBSIDENCE RISK ANALYSIS SUMMARY\n")
        f.write("=" * 50 + "\n\n")
        
        f.write(f"Total Buildings Analyzed: {len(scored_buildings)}\n\n")
        
        f.write("RISK DISTRIBUTION:\n")
        for cat in ['High', 'Medium', 'Low']:
            count = len(scored_buildings[scored_buildings['risk_category'] == cat])
            pct = count / len(scored_buildings) * 100
            f.write(f"  {cat}: {count} ({pct:.1f}%)\n")
        
        f.write("\nSCORE STATISTICS:\n")
        for col in ['risk_score', 'soil_score', 'tree_score']:
            f.write(f"  {col}:\n")
            f.write(f"    Mean: {scored_buildings[col].mean():.2f}\n")
            f.write(f"    Std:  {scored_buildings[col].std():.2f}\n")
            f.write(f"    Min:  {scored_buildings[col].min():.2f}\n")
            f.write(f"    Max:  {scored_buildings[col].max():.2f}\n")
        
        f.write("\nSOIL TYPE DISTRIBUTION:\n")
        for soil, count in scored_buildings['soil_type'].value_counts().items():
            pct = count / len(scored_buildings) * 100
            f.write(f"  {soil}: {count} ({pct:.1f}%)\n")
    
    print(f"✓ Statistics: {stats_file}")
    
    print(f"\n{'=' * 60}")
    print("EXPORT COMPLETE")
    print(f"{'=' * 60}")
    
    return {
        'geojson': geojson_file,
        'shapefile': shp_file,
        'csv': csv_file,
        'statistics': stats_file
    }

# Export results
if 'scored_buildings' in dir() and not scored_buildings.empty:
    exported_files = export_results(scored_buildings)
else:
    print("Run the scoring cell first to export results")

: 

## 7. Summary

This notebook provides a complete workflow for building-level subsidence risk assessment:

✅ **Data Collection** - Trees (Bristol API), Buildings (OSM), Soil (BGS WMS with tiled fetching)

✅ **Risk Scoring** - Per-building scores combining soil type (40%) + tree proximity (60%)

✅ **Spatial Optimization** - R-tree index for fast tree proximity queries (~40x faster)

✅ **Visualization** - Interactive map with toggleable layers and legend

✅ **Export** - GeoJSON with full risk attributes for each building

### Output Files

| File | Description |
|------|-------------|
| `bristol_buildings_scored.geojson` | All buildings with risk scores and attributes |
| `subsidence_risk_buildings.html` | Interactive map (open in browser) |

### Risk Score Interpretation

| Score | Category | Typical Scenario |
|-------|----------|------------------|
| ≥ 7.0 | **High** | Clay soil + multiple trees in root zone |
| 4.0-7.0 | **Medium** | Some risk factors present |
| < 4.0 | **Low** | Sandy/loam soil, few nearby trees |